# Density Profiles at Given Times

This notebook plots a number of density profiles at given times and plots an underlying signal. Both in $\rho$ and in R.

In [ ]:
%matplotlib notebook

import matplotlib
# matplotlib.use("Agg")
from IPython.core.display import display, HTML
from ipfnpytools.getsig import getsig
import ipfnpytools.aug_read as aug_read
from ipfnpytools.closest import closest
from __future__ import print_function
from ipfnpytools.plot import plots
from warnings import warn
import numpy as np
from ipfnpytools.save_figure import save_figure
from ipfnpytools import rps_dump
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import ipywidgets as widgets
from matplotlib.lines import Line2D
import pandas as pd
import reflectometry as rfl
from matplotlib.patches import Polygon

## Parameters

Change these parameters to fit your needs

In [ ]:
# Shot number
shot = 37750

# Choose edition for reflectometry (0 for most recent)
edition = 0

# Choose profile source between 'afs' and 'dump'
source = 'afs'

# Path to dump file 
path = "/afs/ipp-garching.mpg.de/home/d/danielhfc/lab_rot/phd_part8_faitch_initializations/xhfs_2sep-0m050_xlfs_2m230.37765" 

# If chosen 'dump', state the relative path
support_signal = ('FPG', 'delRoben')

# Choose edition for the supporting signal (0 for most recent)
support_signal_edition = 0

# Magnetic equillibrium
equillibrium = "EQH"

# Number of density profiles to show in the plots
number_of_profiles = 3

# Size of window to average profiles [ms]
time_window = 100

# Show error bars corresponding to the standard deviation of the average profile
show_std = True

# Show HFS and LFS rho profiles in the same window
show_same = False

# Show wall
show_wall = True

## Load support signal from AFS

In [ ]:
signal = getsig(shot, support_signal[0], support_signal[1], edition=support_signal_edition)
plots(signal.time, signal.data, labels=signal.name)

## Get density profile data

In [ ]:
# Fetching data ---------------------------------------------------

time, lfs_signal, hfs_signal, lfs_area, hfs_area = rfl.fetch(shot, path if source=='dump' else None, verbose=True)
lfs_signal*=1e-19
hfs_signal*=1e-19

In [ ]:
# Determine which time indexes draw profiles
draw = np.linspace(0, len(time), number_of_profiles + 2, dtype=np.int)
draw = draw[1:-1]

# Handling average profiles
time_limits = np.array([-1.0, 1.0])*time_window/2000
time_limits = np.array([time_limits+time[d] for d in draw])

indexes = closest(time, time_limits.flatten()).reshape((number_of_profiles, 2))
N = indexes[0, 1] - indexes[0, 0]
print("Averaging %d profiles on a %.3lf ms window." % (N, time_window))

# Remove NaN's
lfs_area = np.nan_to_num(lfs_area)
hfs_area = np.nan_to_num(hfs_area)

lfs_average = pd.DataFrame(lfs_area).rolling(window=N).mean().values
hfs_average = pd.DataFrame(hfs_area).rolling(window=N).mean().values

lfs_average = np.nan_to_num(lfs_average)
hfs_average = np.nan_to_num(hfs_average)

lfs_std = pd.DataFrame(lfs_area).rolling(window=N).std().values
hfs_std = pd.DataFrame(hfs_area).rolling(window=N).std().values

lfs_std = np.nan_to_num(lfs_std)
hfs_std = np.nan_to_num(hfs_std)

lfs_low = lfs_average - lfs_std
lfs_up = lfs_average + lfs_std

hfs_low = hfs_average - hfs_std
hfs_up = hfs_average + hfs_std


In [ ]:
fig1, ax1, ax2, ax3 = rfl.two_plus_one(
    x1=signal.time, y1=signal.data, 
    x23=time, y2=hfs_average, y3=lfs_average, 
    z2=hfs_signal, z3=lfs_signal, 
    ey2=hfs_std, ey3=lfs_std, 
    lx1="Time [s]", ly1='',
    ly2="HFS R [m]", lz2=r'$\mathrm{n_{e}\,[10^{19}\,m^{-3}]}$',
    ly3="LFS R [m]", lz3=r'$\mathrm{n_{e}\,[10^{19}\,m^{-3}]}$',
    number_of_profiles=3, 
    m2=None, m3=None, 
    strip_pts=N, show_same=False, 
    paint_rho=False, paint_wall=True,
    sharex=False, sharey=True,
)

# *** PARAMETERS ***
# Plotting options. `None` means automatic

fig1.set_size_inches(10, 7)

ax1.set_xlim(None, None)  # Bottom plot x-range
ax1.set_ylim(None, None)  # Bottom plot y-range

ax2.set_xlim(1, 1.2)  # Top-left plot x-range
ax2.set_ylim(0, 3)  # Top-left plot y-range

ax3.set_xlim(1.6, 2.3) # Top-right plot x-range
ax3.set_ylim(0, 3)  # Top-right plot y-range

In [ ]:
save_figure(fig1, "%d_average_profiles" % (shot))

## Change coordinates to $\rho$

In [ ]:
import dd #To read 
import kk_abock
import numpy as np
from ipfnpytools.trz_to_rhop import trz_to_rhop

In [ ]:
# Set the antenna Z
zl = 0.14
zh = 0.07

# Walls
inner_wall = 1.045
outer_wall = 2.22

inner_wall_rho = trz_to_rhop(time, inner_wall, zh, shot, eq=equillibrium)
outer_wall_rho = trz_to_rhop(time, outer_wall, zl, shot, eq=equillibrium)

rhol = trz_to_rhop(time, lfs_area, np.ones_like(lfs_area)*zl, shot, eq=equillibrium)
rhoh = trz_to_rhop(time, hfs_area, np.ones_like(hfs_area)*zh, shot, eq=equillibrium)

# Compute the average and std in rho ------------------------

rhol_average = pd.DataFrame(rhol).rolling(window=N, center=True).mean().values
rhoh_average = pd.DataFrame(rhoh).rolling(window=N, center=True).mean().values

rhol_std = pd.DataFrame(rhol).rolling(window=N, center=True).std().values
rhoh_std = pd.DataFrame(rhoh).rolling(window=N, center=True).std().values

# Remove NaN's
rhol_average = np.nan_to_num(rhol_average)
rhoh_average = np.nan_to_num(rhoh_average)
rhol_std = np.nan_to_num(rhol_std)
rhol_std = np.nan_to_num(rhol_std)

In [ ]:
fig2 = plt.figure(figsize=(10, 5))
plt.subplots_adjust(hspace=0.6)

# Supporting plot with colored lines
ax4 = plt.subplot(212)
stamp2 = []
ax4.plot(signal.time, signal.data, color='k')
for a, b in indexes:
#     l = ax4.axvline(time[i], color=next(ax4._get_lines.prop_cycler)['color'])
    color = next(ax4._get_lines.prop_cycler)['color']
    l = ax4.axvspan(time[a], time[b], color=color, alpha=0.6)
    stamp2.append(l)

if show_same:
    ax5 = plt.subplot(211)
    ax6 = ax5
else:
    ax5 = plt.subplot(221)
    
    ax6 = plt.subplot(222, sharey=ax5, sharex=ax5)

rhoh = []
hfs_wall = []
rhol = []
lfs_wall = []
   
legend_lines = []
for i in draw:
    color = next(ax5._get_lines.prop_cycler)['color']
    
    legend_lines.append(Line2D([0], [0], color=color, linewidth=1.5, linestyle='-'))
    
    l, = ax5.plot(rhoh_average[i], hfs_signal[i], color=color)
    hfs.append(l)
    
    l, = ax6.plot(rhol_average[i], lfs_signal[i], color=color)
    lfs.append(l)
    
    if show_std:
        p = rfl.fill_between_x(rhoh_average[i] - rhoh_std[i], rhoh_average[i] + rhoh_std[i], hfs_signal[i], color=color, alpha=0.5)
        ax5.add_patch(p)
        rhoh_polygons.append(p)    
        
        p = rfl.fill_between_x(rhol_average[i] - rhol_std[i], rhol_average[i] + rhol_std[i], hfs_signal[i], color=color, alpha=0.5)
        ax6.add_patch(p)
        rhol_polygons.append(p) 
    
#     if show_std:
#         l = ax5.errorbar(rhoh_average[i], hfs_signal[i], xerr=rhoh_std[i], 
#                          alpha=0.5, color=color, label='%.3lf s (HFS)' % time[i])
#     else:
#         l, = ax5.plot(rhoh_average[i], hfs_signal[i], color=color, label='%.3lf s (HFS)' % time[i])
#     rhoh.append(l)
    
#     legend_lines.append(Line2D([0], [0], color=color, linewidth=1.5, linestyle='-'))
    
#     if show_std:
#         l = ax6.errorbar(rhol_average[i], hfs_signal[i], xerr=rhoh_std[i], 
#                          alpha=0.5, color=color, linestyle='--', label='%.3lf s (LFS)' % time[i])
#     else:
#         l, = ax6.plot(rhol_average[i], hfs_signal[i], color=color, label='%.3lf s (LFS)' % time[i])
#     rhol.append(l)
    
    if show_wall:
        l = ax5.axvline(inner_wall_rho[i], color=color, linestyle='-')
        hfs_wall.append(l)
        l = ax6.axvline(outer_wall_rho[i], color=color, linestyle='--')
        lfs_wall.append(l)
        
solid_line = Line2D([0], [0], color='k', linewidth=1.5, linestyle='-')
dashed_line = Line2D([0], [0], color='k', linewidth=1.5, linestyle='--')
        
legend2 = ax6.legend(handles=legend_lines + [solid_line, dashed_line],
                    labels = ["%.3lf s" % time[i] for i in draw] + ['HFS', 'LFS'],
                    loc='center left', bbox_to_anchor=(1, 0, 1, 1))
    
if show_same:
    #labels & text
    ax5.set_ylabel(r'$\mathrm{n_{e}\,[10^{19}\,m^{-3}]}$')
    ax5.set_xlabel(r'$\rho$ poloidal')
    ax5.axvspan(0.0, 1.0, color='#FFC0CB')
#     hfs_legend2 = lfs_legend2
    
else:
    #labels & text
    ax5.set_ylabel(r'$\mathrm{n_{e}\,[10^{19}\,m^{-3}]}$')
    ax5.set_xlabel(r'HFS $\rho$ poloidal')
    ax6.set_xlabel(r'LFS $\rho$ poloidal')
    ax5.axvspan(0.0, 1.0, color='#FFC0CB')
    ax6.axvspan(0.0, 1.0, color='#FFC0CB')


ax4.set_xlabel('Time [s]')

# Default window limits -----------------------------------------
ax5.set_xlim(0.8, 1.2) # In xx direction (rho)
ax5.set_ylim(-0.1, 3)  # In yy direction (density * 1e19)

def update2(**kwargs):
    slider_time = kwargs.values()
    slider_time.sort()
    for i, ts in enumerate(slider_time):
        time_index = closest(time, ts)
        if show_std:
            rfl.update_errorbar(rhoh[i], rhoh_average[time_index], hfs_signal[time_index], xerr=rhoh_std[time_index])
            rfl.update_errorbar(rhol[i], rhol_average[time_index], lfs_signal[time_index], xerr=rhoh_std[time_index])
        else:
            rhoh[i].set_xdata(rhoh_average[time_index])
            rhol[i].set_xdata(rhol_average[time_index])
#         stamp[i].set_xdata(2*[ts])
        stamp2[i].set_xy(
        [[ts-time_window/2000., 0.        ],
         [ts-time_window/2000., 1.        ],
         [ts+time_window/2000., 1.        ],
         [ts+time_window/2000., 0.        ],
         [ts-time_window/2000., 0.        ]]
        )
        if show_wall:
            hfs_wall[i].set_xdata(2*[inner_wall_rho[time_index]])
            lfs_wall[i].set_xdata(2*[outer_wall_rho[time_index]])
        legend2.get_texts()[i].set_text('%.3lf s' % ts)
#         lfs_legend2.get_texts()[2*i+1 if show_same else i].set_text('%.3lf s (LFS)' % ts)
    
    
sliders2 = []
for i in range(number_of_profiles):
    sliders2.append(widgets.FloatSlider(
        value=time[draw[i]],
        min=time[0],
        max=time[-1],
        step=time[1]-time[0],
        description='Time [s]',
        disabled=False,
        continuous_update=True,
        orientation='horizontal',
        readout=True,
        readout_format='.6f',
    ))
    
kwargs = {'p{0}'.format(i):slider for i, slider in enumerate(sliders2)}

widgets.interact(update2, **kwargs);

**Change the window manually, or by running the cell bellow**

In [ ]:
# rho poloidal window limits
ax5.set_xlim(0.9, 1.2) # In xx direction (rho)
ax5.set_ylim(-0.1, 3)  # In yy direction (density * 1e19)

# Support signal window limits
ax4.set_xlim(0, 8)  # In x direction (time)
ax4.set_ylim(-0.1, 1)  # In y direction

In [ ]:
save_figure(fig2, "%d_average_profiles_rho" % (shot))